In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Advanced plotting setup
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

print("🔬 ADVANCED HEATWAVE RESEARCH & PREDICTION ANALYSIS")
print("=" * 60)
print("Loading libraries and setting up advanced analysis environment...")

# Load all your processed data
cpv_raw = pd.read_csv('/home/raj.ayush/results/heatwaves/cpv_new100.csv')
print(f"✅ Raw heatwave data loaded: {len(cpv_raw)} events")

# Load family data
families_data = {}
for fam_id in range(4):
    try:
        families_data[fam_id] = pd.read_csv(f'/home/raj.ayush/results/clustering_step1/cpv_fam{fam_id}.csv')
        families_data[fam_id]['family_id'] = fam_id
        print(f"✅ Family {fam_id} loaded: {len(families_data[fam_id])} events")
    except:
        print(f"⚠️  Family {fam_id} not found")

print("🚀 Ready for advanced analysis!")

In [ ]:
# Advanced feature engineering for prediction
print("🔧 ADVANCED FEATURE ENGINEERING")
print("=" * 40)

def create_advanced_features(df):
    """Create advanced features for heatwave prediction"""
    df_features = df.copy()
    
    # Temporal features
    df_features['time_amin'] = pd.to_datetime(df_features['time_amin'])
    df_features['time_amax'] = pd.to_datetime(df_features['time_amax'])
    
    df_features['year'] = df_features['time_amin'].dt.year
    df_features['month'] = df_features['time_amin'].dt.month
    df_features['day_of_year'] = df_features['time_amin'].dt.dayofyear
    df_features['season'] = ((df_features['month'] - 1) // 3) + 1
    
    # Cyclical encoding for seasonal patterns
    df_features['month_sin'] = np.sin(2 * np.pi * df_features['month'] / 12)
    df_features['month_cos'] = np.cos(2 * np.pi * df_features['month'] / 12)
    df_features['day_sin'] = np.sin(2 * np.pi * df_features['day_of_year'] / 365)
    df_features['day_cos'] = np.cos(2 * np.pi * df_features['day_of_year'] / 365)
    
    # Spatial features
    df_features['lat_lon_interaction'] = df_features['latitude_mean'] * df_features['longitude_mean']
    df_features['spatial_distance_from_center'] = np.sqrt(
        (df_features['latitude_mean'] - df_features['latitude_mean'].mean())**2 + 
        (df_features['longitude_mean'] - df_features['longitude_mean'].mean())**2
    )
    
    # Advanced heatwave characteristics
    df_features['intensity_per_day'] = df_features['HWMId_magnitude'] / (df_features['timespan'] + 1)
    df_features['magnitude_log'] = np.log1p(df_features['HWMId_magnitude'])
    df_features['duration_log'] = np.log1p(df_features['timespan'])
    
    # Statistical features
    df_features['magnitude_zscore'] = (df_features['HWMId_magnitude'] - df_features['HWMId_magnitude'].mean()) / df_features['HWMId_magnitude'].std()
    df_features['duration_zscore'] = (df_features['timespan'] - df_features['timespan'].mean()) / df_features['timespan'].std()
    
    # Temporal trends
    df_features['years_since_start'] = df_features['year'] - df_features['year'].min()
    
    return df_features

# Apply feature engineering to raw data
cpv_enhanced = create_advanced_features(cpv_raw)

# Filter clean data
cpv_clean = cpv_enhanced[
    (cpv_enhanced['timespan'] > 0) & 
    (cpv_enhanced['timespan'] <= 365) & 
    (cpv_enhanced['HWMId_magnitude'] > 0)
].copy()

print(f"✅ Enhanced dataset created: {len(cpv_clean)} events with {len(cpv_clean.columns)} features")

# Display new features
new_features = ['month_sin', 'month_cos', 'day_sin', 'day_cos', 'lat_lon_interaction', 
                'spatial_distance_from_center', 'intensity_per_day', 'magnitude_log', 
                'duration_log', 'magnitude_zscore', 'duration_zscore', 'years_since_start']

print("🎯 New engineered features:")
for feat in new_features:
    print(f"   • {feat}")

In [ ]:
# Advanced trend analysis
print("📈 ADVANCED TREND ANALYSIS & FORECASTING")
print("=" * 45)

# Yearly aggregation for trend analysis
yearly_analysis = cpv_clean.groupby('year').agg({
    'HWMId_magnitude': ['count', 'mean', 'std', 'max'],
    'timespan': ['mean', 'std', 'max'],
    'intensity_per_day': ['mean', 'std'],
    'spatial_distance_from_center': 'mean'
}).round(3)

yearly_analysis.columns = ['_'.join(col).strip() for col in yearly_analysis.columns]
yearly_analysis = yearly_analysis.reset_index()

print("📊 Yearly trend analysis:")
display(yearly_analysis.head())

# Advanced trend visualization
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('🔬 Advanced Heatwave Trend Analysis', fontsize=16, fontweight='bold')

# 1. Frequency trend with confidence intervals
years = yearly_analysis['year']
event_counts = yearly_analysis['HWMId_magnitude_count']
z = np.polyfit(years, event_counts, 1)
p = np.poly1d(z)
trend_line = p(years)

axes[0,0].scatter(years, event_counts, alpha=0.7, s=60, color='darkblue')
axes[0,0].plot(years, trend_line, '--', color='red', linewidth=2, label=f'Trend: {z[0]:+.2f} events/year')
axes[0,0].fill_between(years, trend_line - np.std(event_counts - trend_line), 
                       trend_line + np.std(event_counts - trend_line), alpha=0.2, color='red')
axes[0,0].set_title('📊 Heatwave Frequency Trend')
axes[0,0].set_xlabel('Year')
axes[0,0].set_ylabel('Number of Events')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Intensity evolution
mean_intensity = yearly_analysis['HWMId_magnitude_mean']
z2 = np.polyfit(years, mean_intensity, 1)
p2 = np.poly1d(z2)

axes[0,1].scatter(years, mean_intensity, alpha=0.7, s=60, color='darkred')
axes[0,1].plot(years, p2(years), '--', color='orange', linewidth=2, label=f'Trend: {z2[0]:+.2f}/year')
axes[0,1].set_title('🌡️  Mean Intensity Trend')
axes[0,1].set_xlabel('Year')
axes[0,1].set_ylabel('Mean Magnitude')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Duration evolution
mean_duration = yearly_analysis['timespan_mean']
z3 = np.polyfit(years, mean_duration, 1)
p3 = np.poly1d(z3)

axes[0,2].scatter(years, mean_duration, alpha=0.7, s=60, color='darkgreen')
axes[0,2].plot(years, p3(years), '--', color='lime', linewidth=2, label=f'Trend: {z3[0]:+.2f} days/year')
axes[0,2].set_title('⏱️  Mean Duration Trend')
axes[0,2].set_xlabel('Year')
axes[0,2].set_ylabel('Mean Duration (days)')
axes[0,2].legend()
axes[0,2].grid(True, alpha=0.3)

# 4. Extreme events over time
extreme_threshold = cpv_clean['HWMId_magnitude'].quantile(0.95)
yearly_extremes = cpv_clean[cpv_clean['HWMId_magnitude'] >= extreme_threshold].groupby('year').size()
all_years = range(cpv_clean['year'].min(), cpv_clean['year'].max() + 1)
yearly_extremes = yearly_extremes.reindex(all_years, fill_value=0)

axes[1,0].bar(yearly_extremes.index, yearly_extremes.values, alpha=0.7, color='purple')
z4 = np.polyfit(yearly_extremes.index, yearly_extremes.values, 1)
p4 = np.poly1d(z4)
axes[1,0].plot(yearly_extremes.index, p4(yearly_extremes.index), '--', color='red', linewidth=2)
axes[1,0].set_title('🚨 Extreme Events Trend (Top 5%)')
axes[1,0].set_xlabel('Year')
axes[1,0].set_ylabel('Number of Extreme Events')
axes[1,0].grid(True, alpha=0.3)

# 5. Seasonal intensity patterns
seasonal_intensity = cpv_clean.groupby('month')['HWMId_magnitude'].agg(['mean', 'std']).reset_index()
months = seasonal_intensity['month']
mean_vals = seasonal_intensity['mean']
std_vals = seasonal_intensity['std']

axes[1,1].errorbar(months, mean_vals, yerr=std_vals, marker='o', linewidth=2, capsize=5)
axes[1,1].fill_between(months, mean_vals - std_vals, mean_vals + std_vals, alpha=0.3)
axes[1,1].set_title('🗓️  Seasonal Intensity Patterns')
axes[1,1].set_xlabel('Month')
axes[1,1].set_ylabel('Mean Magnitude ± Std')
axes[1,1].set_xticks(range(1, 13))
axes[1,1].grid(True, alpha=0.3)

# 6. Spatial-temporal correlation
yearly_spatial = yearly_analysis[['year', 'spatial_distance_from_center_mean']]
axes[1,2].scatter(yearly_spatial['year'], yearly_spatial['spatial_distance_from_center_mean'], 
                  alpha=0.7, s=60, color='brown')
z5 = np.polyfit(yearly_spatial['year'], yearly_spatial['spatial_distance_from_center_mean'], 1)
p5 = np.poly1d(z5)
axes[1,2].plot(yearly_spatial['year'], p5(yearly_spatial['year']), '--', color='orange', linewidth=2)
axes[1,2].set_title('🗺️  Spatial Dispersion Trend')
axes[1,2].set_xlabel('Year')
axes[1,2].set_ylabel('Mean Distance from Center')
axes[1,2].grid(True, alpha=0.3)

plt.tight_layout()
# plt.savefig('/home/raj.ayush/results/final_ananananal/advanced_trend_analysis.png')
plt.show()

# Statistical significance of trends
print("📈 TREND SIGNIFICANCE ANALYSIS:")
trends_data = [
    ('Frequency', z[0], event_counts, years),
    ('Intensity', z2[0], mean_intensity, years),
    ('Duration', z3[0], mean_duration, years),
    ('Extremes', z4[0], yearly_extremes.values, yearly_extremes.index)
]

for name, slope, y_data, x_data in trends_data:
    correlation, p_value = stats.pearsonr(x_data, y_data)
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
    print(f"   • {name}: {slope:+.4f}/year, r={correlation:.3f}, p={p_value:.4f} {significance}")

In [ ]:
# Advanced machine learning for heatwave prediction
print("🤖 MACHINE LEARNING PREDICTION MODELS")
print("=" * 40)

# Prepare data for ML
feature_columns = [
    'latitude_mean', 'longitude_mean', 'n_unique_g_ids', 'year', 'month', 'day_of_year',
    'month_sin', 'month_cos', 'day_sin', 'day_cos', 'lat_lon_interaction',
    'spatial_distance_from_center', 'years_since_start', 'season'
]

# Prediction tasks
prediction_tasks = {
    'Duration': 'timespan',
    'Magnitude': 'HWMId_magnitude', 
    'Intensity': 'intensity_per_day'
}

ml_results = {}

for task_name, target_col in prediction_tasks.items():
    print(f"\n🎯 Predicting {task_name}...")
    
    # Prepare data
    X = cpv_clean[feature_columns].copy()
    y = cpv_clean[target_col].copy()
    
    # Handle any missing values
    X = X.fillna(X.mean())
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Models to test
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42)
    }
    
    task_results = {}
    
    for model_name, model in models.items():
        # Train model
        if 'XGBoost' in model_name or 'Forest' in model_name or 'Gradient' in model_name:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        else:
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        
        # Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Cross-validation score
        if 'XGBoost' in model_name or 'Forest' in model_name or 'Gradient' in model_name:
            cv_score = cross_val_score(model, X_train, y_train, cv=5, scoring='r2').mean()
        else:
            cv_score = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2').mean()
        
        task_results[model_name] = {
            'RMSE': rmse,
            'MAE': mae,
            'R²': r2,
            'CV_R²': cv_score,
            'predictions': y_pred,
            'actual': y_test
        }
        
        print(f"   {model_name:15}: R²={r2:.3f}, RMSE={rmse:.2f}, CV_R²={cv_score:.3f}")
    
    ml_results[task_name] = task_results

print("\n✅ Machine learning models trained successfully!")

In [ ]:
# Visualize ML model performance
print("📊 MODEL PERFORMANCE VISUALIZATION")
print("=" * 35)

fig, axes = plt.subplots(3, 3, figsize=(20, 18))
fig.suptitle('🤖 Machine Learning Model Performance Analysis', fontsize=16, fontweight='bold')

for task_idx, (task_name, task_results) in enumerate(ml_results.items()):
    # Performance metrics comparison
    models = list(task_results.keys())
    r2_scores = [task_results[model]['R²'] for model in models]
    cv_scores = [task_results[model]['CV_R²'] for model in models]
    rmse_scores = [task_results[model]['RMSE'] for model in models]
    
    # R² comparison
    x_pos = np.arange(len(models))
    axes[task_idx, 0].bar(x_pos - 0.2, r2_scores, 0.4, label='Test R²', alpha=0.8, color='skyblue')
    axes[task_idx, 0].bar(x_pos + 0.2, cv_scores, 0.4, label='CV R²', alpha=0.8, color='orange')
    axes[task_idx, 0].set_title(f'{task_name}: R² Score Comparison')
    axes[task_idx, 0].set_ylabel('R² Score')
    axes[task_idx, 0].set_xticks(x_pos)
    axes[task_idx, 0].set_xticklabels(models, rotation=45, ha='right')
    axes[task_idx, 0].legend()
    axes[task_idx, 0].grid(True, alpha=0.3)
    
    # RMSE comparison
    axes[task_idx, 1].bar(models, rmse_scores, alpha=0.8, color='lightcoral')
    axes[task_idx, 1].set_title(f'{task_name}: RMSE Comparison')
    axes[task_idx, 1].set_ylabel('RMSE')
    axes[task_idx, 1].tick_params(axis='x', rotation=45)
    axes[task_idx, 1].grid(True, alpha=0.3)
    
    # Best model prediction vs actual
    best_model = max(task_results.keys(), key=lambda x: task_results[x]['R²'])
    y_pred = task_results[best_model]['predictions']
    y_actual = task_results[best_model]['actual']
    
    axes[task_idx, 2].scatter(y_actual, y_pred, alpha=0.6, s=30)
    # Perfect prediction line
    min_val = min(min(y_actual), min(y_pred))
    max_val = max(max(y_actual), max(y_pred))
    axes[task_idx, 2].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    axes[task_idx, 2].set_title(f'{task_name}: Best Model ({best_model})\nR² = {task_results[best_model]["R²"]:.3f}')
    axes[task_idx, 2].set_xlabel('Actual Values')
    axes[task_idx, 2].set_ylabel('Predicted Values')
    axes[task_idx, 2].legend()
    axes[task_idx, 2].grid(True, alpha=0.3)

plt.tight_layout()
# plt.savefig('/home/raj.ayush/results/final_ananananal/ml_model_performance.png')
plt.show()

# Performance summary table
print("📈 MODEL PERFORMANCE SUMMARY TABLE")
performance_summary = []
for task_name, task_results in ml_results.items():
    for model_name, results in task_results.items():
        performance_summary.append({
            'Task': task_name,
            'Model': model_name,
            'R²': results['R²'],
            'RMSE': results['RMSE'],
            'MAE': results['MAE'],
            'CV_R²': results['CV_R²']
        })

performance_df = pd.DataFrame(performance_summary)
print("\n🏆 Best performing models:")
for task in prediction_tasks.keys():
    task_df = performance_df[performance_df['Task'] == task]
    best_model = task_df.loc[task_df['R²'].idxmax()]
    print(f"   {task:12}: {best_model['Model']:15} (R²={best_model['R²']:.3f}, RMSE={best_model['RMSE']:.2f})")

display(performance_df.round(3))